In [4]:
%load_ext sql

In [2]:
%sql sqlite:///smaller.db
%sql select * from data limit 5

Done.


state,name,orispl_code,unitid,op_date,year,op_hour,op_time,gload,SO2_MASS,NOX_MASS,CO2_MASS,heat_input,county,latitude,longitude,fuel_type,fuel_type_2,fuel_actual
WA,Centralia,3845,BW21,01-01-2001,2001,0,1.0,704.0,11493.6,2506.854,720.5,7022.0,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,1,1.0,706.0,11774.7,2577.369,736.6,7179.3,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,2,1.0,704.0,11156.9,2606.306,718.8,7006.2,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,3,1.0,702.0,10990.0,2561.697,716.2,6980.1,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,4,1.0,702.0,11027.6,2631.762,716.2,6980.8,Lewis,46.7559,-122.8578,Coal,,Coal


In [4]:
%sql sqlite:///coal.db
%sql drop table if exists data 
%sql vacuum

(sqlite3.OperationalError) no such table: data [SQL: u'drop table data']
Done.


[]

In [5]:
%%sql
sqlite:///smaller.db
ATTACH DATABASE 'coal.db' AS coal;
create table coal.data as select * from data where fuel_type is 'Coal';

Done.
Done.


[]

In [11]:
%sql select *, CO2_MASS / heat_input as CO2_rate from data where CO2_rate < 0.07 and CO2_rate > 0.05 group by orispl_code, unitid order by state

Done.


state,facility_name,orispl_code,unitid,op_date,op_hour,op_time,gload,SO2_MASS,NOX_MASS,CO2_MASS,HEAT_INPUT,state:1,facility_name:1,orispl_code:1,unitid:1,year,county,latitude,longitude,fuel_type,fule_type_secondary,fuel_actual,CO2_rate
AZ,Ocotillo Power Plant,116,1,12-17-2016,22,0.3,37.0,0.072,13.028,7.11,119.52,AZ,Ocotillo Power Plant,116,1,2016,Maricopa,33.4225,-111.9122,Pipeline Natural Gas,Residual Oil,NG,0.0594879518072
AZ,Ocotillo Power Plant,116,2,12-12-2016,22,0.17,16.0,0.025,3.847,2.516,42.279,AZ,Ocotillo Power Plant,116,2,2016,Maricopa,33.4225,-111.9122,Pipeline Natural Gas,Residual Oil,NG,0.0595094491355
AZ,APS West Phoenix Power Plant,117,CC4,12-09-2016,23,0.42,39.0,0.092,19.416,9.072,152.88,AZ,APS West Phoenix Power Plant,117,CC4,2016,Maricopa,33.4417,-112.1583,Pipeline Natural Gas,,NG,0.0593406593407
AZ,APS West Phoenix Power Plant,117,CC5A,12-31-2016,23,1.0,156.0,0.743,22.284,73.6,1238.0,AZ,APS West Phoenix Power Plant,117,CC5A,2016,Maricopa,33.4417,-112.1583,Pipeline Natural Gas,,NG,0.059450726979
AZ,APS West Phoenix Power Plant,117,CC5B,12-31-2016,23,1.0,157.0,0.746,22.387,73.9,1243.7,AZ,APS West Phoenix Power Plant,117,CC5B,2016,Maricopa,33.4417,-112.1583,Pipeline Natural Gas,,NG,0.0594194741497
AZ,APS Saguaro Power Plant,118,1,09-15-2011,17,0.45,15.0,0.06,7.935,5.985,100.44,AZ,APS Saguaro Power Plant,118,1,2011,Pinal,32.5517,-111.3,Pipeline Natural Gas,Residual Oil,NG,0.0595878136201
AZ,APS Saguaro Power Plant,118,2,09-29-2011,18,0.22,0.0,0.008,0.468,0.814,13.75,AZ,APS Saguaro Power Plant,118,2,2011,Pinal,32.5517,-111.3,Pipeline Natural Gas,Residual Oil,NG,0.0592
AZ,APS Saguaro Power Plant,118,CT3,12-02-2016,19,0.17,19.0,0.076,8.436,3.774,63.427,AZ,APS Saguaro Power Plant,118,CT3,2016,Pinal,32.5517,-111.3,"Natural Gas, Pipeline Natural Gas",,NG,0.0595014741356
AZ,Yucca Power Plant,120,1,09-26-2016,15,0.77,14.0,0.073,10.767,7.238,122.353,AZ,Yucca Power Plant,120,1,2016,Yuma,32.7214,-114.7097,Pipeline Natural Gas,Residual Oil,NG,0.0591567023285
AZ,Yucca Power Plant,120,CT5,12-31-2016,14,0.33,24.0,0.059,0.493,5.874,98.604,AZ,Yucca Power Plant,120,CT5,2016,Yuma,32.7214,-114.7097,Pipeline Natural Gas,,NG,0.0595716198126


In [12]:
%%sql
select name, unitid, max(CAST(gload as float)) as max_gen, sum(cast(gload as float) / cast(heat_input as float)) as capacity_factor, year
from data group by orispl_code, unitid, year limit 5

Done.


name,unitid,max_gen,capacity_factor,year
Escalante,1,260.0,669.395411888,2001
Escalante,1,262.0,838.386143622,2002
Escalante,1,262.0,974.621601526,2003
Escalante,1,264.0,858.907640037,2004
Prewitt Escalante Generating Station,1,265.0,881.917324066,2005


In [6]:
%%sql
sqlite:///coal.db
drop table if exists cf;
create table cf as 
select name, d.orispl_code, d.unitid, state, op_date, op_hour, gload, 
cast(gload as float) / cast(max_gen as float) as capacity_factor
from (select orispl_code, unitid, max(CAST(gload as float)) as max_gen
from data group by orispl_code, unitid) as m, data as d
where d.orispl_code = m.orispl_code and d.unitid = m.unitid;
select * from cf limit 5;

Done.
Done.
Done.


name,orispl_code,unitid,state,op_date,op_hour,gload,capacity_factor
Centralia,3845,BW21,WA,01-01-2001,0,704.0,0.907216494845
Centralia,3845,BW21,WA,01-01-2001,1,706.0,0.909793814433
Centralia,3845,BW21,WA,01-01-2001,2,704.0,0.907216494845
Centralia,3845,BW21,WA,01-01-2001,3,702.0,0.904639175258
Centralia,3845,BW21,WA,01-01-2001,4,702.0,0.904639175258


In [32]:
%%sql
select count(*), substr(op_date, -4) as year from cf where capacity_factor > 0 and capacity_factor <= 0.1 group by year

Done.


count(*),year
2534,2001
2412,2002
2265,2003
2383,2004
2307,2005
2167,2006
2344,2007
2501,2008
2793,2009
3872,2010


In [1]:
import sqlite3
import pandas
import os.path
con = sqlite3.connect('coal.db')
labels = pandas.read_sql("select orispl_code, unitid from data group by orispl_code, unitid", con)


In [2]:
# Generates csv file for each unit, for each year, of capacity factors.
if not os.path.exists('cf'):
    os.makedirs('cf')
for i, row in labels.iterrows():
    print row['orispl_code'], row['unitid']
    for year in range(2001, 2017):
        file_name = 'cf/' + str(row['orispl_code']) + '_' + row['unitid'] + '_' + str(year) + '.csv'
        if os.path.isfile(file_name): continue
        table = pandas.read_sql("select name, orispl_code, unitid, op_date, op_hour, capacity_factor from cf\
        where orispl_code is " + str(row['orispl_code']) + " and unitid is '" + row['unitid'] + "' and \
        substr(op_date, -4) is '" + str(year) + "'", con)
        table.to_csv(file_name)

87 1
113 1
113 2
113 3
113 4
126 4
160 2
160 3
465 1
465 2
465 3
465 4
468 2
469 1
469 2
469 3
469 4
470 1
470 2
470 3
477 5
492 5
492 6
492 7
508 8
525 H1
525 H2
527 1
2187 2
2324 1
2324 2
2324 3
2324 4
2341 1
2341 2
2442 1
2442 2
2442 3
2442 4
2442 5
2451 1
2451 2
2451 3
2451 4
3644 1
3644 2
3845 BW21
3845 BW22
4158 BW41
4158 BW42
4158 BW43
4158 BW44
4162 1
4162 2
4162 3
4941 1
4941 2
4941 3
6021 C1
6021 C2
6021 C3
6076 1
6076 2
6076 3
6076 4
6089 B1
6101 BW91
6106 1SG
6165 1
6165 2
6165 3
6177 U1B
6177 U2B
6204 1
6204 2
6204 3
6248 1
6481 1SGA
6481 2SGA
6761 101
7504 001
7790 1-1
8066 BW71
8066 BW72
8066 BW73
8066 BW74
8069 1
8069 2
8219 1
8223 1
8223 2
8223 4
8223 TS3
8224 1
8224 2
50951 1
55479 001
55749 U1
56224 001
56319 001
56596 001
56609 01


In [3]:
%sql sqlite:///coal.db 
%sql select count(*) from data where fuel_type is 'Coal' and year is 2016 group by name, unitid            

ERROR:root:Line magic function `%sql` not found.
ERROR:root:Line magic function `%sql` not found.


In [12]:
%%sql
sqlite:///coal.db
delete from data where state is 'TX' and (county is not 'El Paso' or name is 'Montana Power Station');
vacuum;

5386833 rows affected.
Done.


[]

In [16]:
%sql select * from data where state is 'TX'

Done.


state,name,orispl_code,unitid,op_date,year,op_hour,op_time,gload,SO2_MASS,NOX_MASS,CO2_MASS,heat_input,county,latitude,longitude,fuel_type,fuel_type_2,fuel_actual
